In [8]:
import os
from copy import deepcopy
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from rdkit.Chem import rdFingerprintGenerator
from rdkit import Chem
from tqdm.contrib.concurrent import thread_map

datasets = {
    'c-binding': 'Protein-peptide binding affinity (canonical)',
    'binding': 'Protein-peptide binding affinity',
    'c-cpp': 'Cell penetration (canonical)',
    'cpp': 'Cell penetration',
    'nc-binding': 'Protein-peptide binding affinity (non-canonical)',
    'nc-cpp': 'Cell penetration (non-canonical)',
    'nc-antibacterial': "Antibacterial (non-canonical)",
    'antibacterial': "Antibacterial",
    'c-antibacterial': "Antibacterial (canonical)",
    'nc-antiviral': "Antiviral (non-canonical)",
    'antiviral': "Antiviral",
    'c-antiviral': "Antiviral (canonical)"
}
datasets_sim = [
    'c-binding', 'nc-binding', 'c-cpp',
    'nc-cpp', 'c-antibacterial', 'nc-antibacterial',
    'c-antiviral', 'nc-antiviral'
]
metrics_class = {
    'acc': 'Accuracy', 'f1_weighted': 'Weighted F1',
    'mcc': 'Matthew\'s correlation coefficient',
    'auroc': "Area under the ROC curve"
}
metrics_reg = {
    'pcc': 'Pearson\'s R', 'spcc': 'Spearman\'s R',
    'rmse': 'RMSE'
}
fancy_rep = {
    'esm2-8m': 'ESM2 8M',
    'esm2-150m': "ESM2 150M",
    'prot-t5-xl': "Prot-T5-XL",
    'molformer': 'Molformer-XL',
    'chemberta': 'ChemBERTa-2',
    'pepfunn': 'PepFuNN',
    'ecfp': "ECFP-16",
    'ecfp-count': "ECFP-16 count",
    'pepclm': "PeptideCLM",
    'pepland': "Pepland",
    ## Add new representations
    'ecfp-count': "ECFP-16 with counts"
}
metrics_fancy = metrics_class 
metrics_fancy.update(metrics_reg)
REGRESSION = ['c-binding', 'nc-binding']
DPI = 512
sorted(datasets_sim)

['c-antibacterial',
 'c-antiviral',
 'c-binding',
 'c-cpp',
 'nc-antibacterial',
 'nc-antiviral',
 'nc-binding',
 'nc-cpp']

In [9]:
from scipy.stats import kruskal, wilcoxon


def get_stats(df: pd.DataFrame, order: list):
    g_df = df.groupby('rep')
    groups = {}

    for n, mini_df in g_df:
        groups[n] = mini_df['GOOD'].to_numpy()

    names = {name: idx for idx, name in enumerate(order)}
    mtx = np.ones((len(groups), len(groups)))

    for pair1 in order:
        for pair2 in order:
            if pair1 == pair2:
                continue    
            if len(groups[pair1]) != len(groups[pair2]):
                value = 2
            else:
                value = wilcoxon(
                    groups[pair1], groups[pair2],
                    alternative='greater'
                )[1]
            mtx[names[pair1], names[pair2]] = value
    return groups, mtx

def order_datasets(df: pd.DataFrame) -> list:
    names = [n for n, g in df.groupby('rep')]
    means = df.groupby('rep')['GOOD'].mean()
    order = np.argsort(means)
    names = [names[i] for i in order]
    return names

def define_table(df: pd.DataFrame) -> pd.DataFrame:
    df['rep'] = df.rep.map(fancy_rep)
    df = df[df['threshold'] != 'random']

    all_proto_df = []
    for idx, dataset in enumerate(df.dataset.unique()):
        tmp_df = df[df['dataset'] == dataset].copy()
        if not dataset.startswith('c-') and not dataset.startswith('nc-'):
            metric = 'GOOD'
        else:
            if dataset in REGRESSION:
                metric = 'spcc'
            else:
                metric = 'mcc'

        new_df = []
        for model in tmp_df.model.unique():
            for threshold in tmp_df.threshold.unique():
                for rep in tmp_df.rep.unique():
                    tmp_df_2 = df[(df.model == model) & (df.threshold == threshold) & (df.rep == rep)]
                    if len(tmp_df_2) < 1:
                        continue
                    new_df.append(
                        {'rep': rep, 'threshold': threshold,
                         'model': model, 'GOOD': tmp_df_2[metric].mean(),
                         'dataset': dataset}
                    )
                    all_proto_df.append(new_df[-1])

    df = pd.DataFrame(all_proto_df)
    names = order_datasets(df)
    groups, mtx = get_stats(df, names)
    # Kruskal-Wallis test
    p_value = kruskal(*list(groups.values()))[1]
    print(f"Kruskal-Wallis p: {p_value:.1g}")

    new_df = []
    n = len(groups)
    alpha_adj = 0.05 * 2 / (n * (n - 1))
    ranks = []
    names.reverse()
    for idx, name in enumerate(names):
        if idx == 0:
            ranks.append(1)
        else:
            if mtx[idx, idx - 1] < alpha_adj:
                ranks.append(ranks[-1])
            else:
                ranks.append(ranks[-1] + 1)

    final_table = []
    for name, rank in zip(names, ranks):
        entry = {"Representation": name}
        for dataset in df.dataset.unique():
            tmp_df = df[(df['rep'] == name) & (df['dataset'] == dataset)]
            std = f"{tmp_df['GOOD'].std():.1g}"
            if len(std) == 4:
                mean = f"{tmp_df['GOOD'].mean():.2f}"
            elif len(std) == 3:
                mean = f"{tmp_df['GOOD'].mean():.1f}"
            else:
                mean = f"{tmp_df['GOOD'].mean():.3g}"

            entry[datasets[dataset]] = f"{mean}±{std}"
    
        tmp_df = df[df['rep'] == name]
        std = f"{tmp_df['GOOD'].std():.1g}"
        if len(std) == 4:
            mean = f"{tmp_df['GOOD'].mean():.2f}"
        elif len(std) == 3:
            mean = f"{tmp_df['GOOD'].mean():.1f}"
        else:
            mean = f"{tmp_df['GOOD'].mean():.3g}"
        entry['Average'] = f"{mean}±{std}"
        entry["Significant rank"] = rank
        final_table.append(entry)

    table = pd.DataFrame(final_table)
    return table

In [10]:
dir = '../Results/no-gen'
df = pd.DataFrame()

for file in os.listdir(dir):

    path = os.path.join(dir, file)
    tmp_df = pd.read_csv(path)
    tmp_df['model'] = file.split('_')[1]
    tmp_df['pre_pca'] = float(file.split('_')[3])
    tmp_df['post_pca'] = float(file.split('_')[5])
    tmp_df['dataset'] = file.split('_')[0]
    tmp_df['rep'] = file.split('_')[6][:-4]
    df = pd.concat([df, tmp_df])

In [11]:
# Canonical LightGBM
print("Values for LightGBM Canonical")
df_c = df[df['dataset'].map(lambda x: x.startswith('c-'))].copy()
df_c = df_c[df_c['model'] == 'lightgbm'].copy()
tab1 = define_table(df_c)
tab1

Values for LightGBM Canonical
Kruskal-Wallis p: 0.02


,Representation,Antiviral (canonical),Protein-peptide binding affinity (canonical),Antibacterial (canonical),Cell penetration (canonical),Average,Significant rank
0,ESM2 8M,0.8±0.1,0.90±0.04,0.84±0.08,0.84±0.08,0.85±0.09,1
1,Prot-T5-XL,0.8±0.1,0.90±0.03,0.84±0.09,0.84±0.09,0.84±0.09,1
2,ESM2 150M,0.8±0.1,0.88±0.04,0.84±0.09,0.84±0.09,0.83±0.09,2
3,ECFP-16,0.8±0.1,0.90±0.04,0.8±0.1,0.8±0.1,0.8±0.1,3
4,ChemBERTa-2,0.8±0.1,0.89±0.04,0.82±0.09,0.82±0.09,0.8±0.1,3
5,PeptideCLM,0.8±0.1,0.86±0.03,0.8±0.1,0.8±0.1,0.8±0.1,4
6,Pepland,0.7±0.1,0.89±0.05,0.8±0.1,0.8±0.1,0.8±0.1,5
7,Molformer-XL,0.7±0.1,0.88±0.05,0.8±0.1,0.8±0.1,0.8±0.1,6
8,PepFuNN,0.7±0.1,0.76±0.04,0.8±0.1,0.8±0.1,0.76±0.09,6


In [12]:
# Non-canonical LightGBM
print("Values for LightGBM Non-canonical")
df_nc = df[df['dataset'].map(lambda x: x.startswith('nc-'))].copy()
df_nc = df_nc[df_nc['model'] == 'lightgbm']
tab2 = define_table(df_nc)
tab2

Values for LightGBM Non-canonical
Kruskal-Wallis p: 5e-12


,Representation,Antiviral (non-canonical),Antibacterial (non-canonical),Protein-peptide binding affinity (non-canonical),Cell penetration (non-canonical),Average,Significant rank
0,Molformer-XL,0.88±0.07,0.91±0.04,0.8±0.1,0.90±0.05,0.88±0.08,1
1,ChemBERTa-2,0.87±0.06,0.89±0.05,0.88±0.09,0.88±0.06,0.88±0.07,2
2,ECFP-16,0.82±0.06,0.84±0.05,0.9±0.1,0.83±0.06,0.84±0.07,2
3,PeptideCLM,0.82±0.05,0.83±0.04,0.9±0.1,0.83±0.04,0.83±0.06,3
4,Pepland,0.7±0.1,0.77±0.05,0.83±0.08,0.7±0.1,0.77±0.09,3
5,PepFuNN,0.73±0.06,0.76±0.04,0.7±0.2,0.74±0.06,0.74±0.09,4


In [13]:
# Canonical to non-canonical
print("Values for generalizing from canonical to non-canonical")
dir = '../Results/canonical'
data, data_2 = [], []

for file in os.listdir(dir):

    dataset = file.split('_')[0]
    rep = file.split('_')[6][:-4]
    path = os.path.join(dir, file)
    experiment = dir.split("/")[-1][0].upper() + dir.split("/")[-1][1:]
    model = file.split('_')[1]
    if model != 'lightgbm':
        continue

    tmp_df = pd.read_csv(path)
    tmp_df['model'] = file.split('_')[1]
    tmp_df['pre_pca'] = float(file.split('_')[3])
    tmp_df['post_pca'] = float(file.split('_')[5])
    tmp_df['dataset'] = dataset
    tmp_df['rep'] = rep
    tmp_df['experiment'] = experiment
    tmp_df = tmp_df[tmp_df['threshold'] != 'random']
    if 'binding' in dataset:
        metric = 'spcc'
    else:
        metric = 'mcc'

    for m, th in zip(tmp_df[f'{metric}_nc'], tmp_df['threshold']):
        data_2.append({
            'dataset': dataset,
            "GOOD": m,
            "Test set": "Non-canonical",
            "model": "lightgbm",
            "rep": rep,
            'threshold': th

        })

df = pd.DataFrame(data_2)
# df.head()
tab3 = define_table(df)
tab3

Values for generalizing from canonical to non-canonical
Kruskal-Wallis p: 3e-22


,Representation,Protein-peptide binding affinity,Antiviral,Antibacterial,Cell penetration,Average,Significant rank
0,ChemBERTa-2,0.23±0.06,0.23±0.06,0.21±0.02,0.21±0.02,0.22±0.05,1
1,ECFP-16,0.18±0.07,0.18±0.07,0.21±0.04,0.21±0.04,0.19±0.06,2
2,PeptideCLM,0.19±0.04,0.19±0.04,0.19±0.04,0.19±0.04,0.19±0.04,3
3,Molformer-XL,0.16±0.02,0.16±0.02,0.15±0.02,0.15±0.02,0.15±0.02,3
4,ECFP-16 with counts,0.13±0.04,0.13±0.04,0.13±0.04,0.13±0.04,0.13±0.04,3
5,PepFuNN,0.09±0.09,0.09±0.09,0.14±0.03,0.14±0.03,0.11±0.07,4
6,Pepland,0.09±0.04,0.09±0.04,0.08±0.04,0.08±0.04,0.09±0.04,5


In [15]:
print(tab1.to_markdown(index=False))

| Representation   | Antiviral (canonical)   | Protein-peptide binding affinity (canonical)   | Antibacterial (canonical)   | Cell penetration (canonical)   | Average   |   Significant rank |
|:-----------------|:------------------------|:-----------------------------------------------|:----------------------------|:-------------------------------|:----------|-------------------:|
| ESM2 8M          | 0.8±0.1                 | 0.90±0.04                                      | 0.84±0.08                   | 0.84±0.08                      | 0.85±0.09 |                  1 |
| Prot-T5-XL       | 0.8±0.1                 | 0.90±0.03                                      | 0.84±0.09                   | 0.84±0.09                      | 0.84±0.09 |                  1 |
| ESM2 150M        | 0.8±0.1                 | 0.88±0.04                                      | 0.84±0.09                   | 0.84±0.09                      | 0.83±0.09 |                  2 |
| ECFP-16          | 0.8±0.1            

In [16]:
print(tab2.to_markdown(index=False))

| Representation   | Antiviral (non-canonical)   | Antibacterial (non-canonical)   | Protein-peptide binding affinity (non-canonical)   | Cell penetration (non-canonical)   | Average   |   Significant rank |
|:-----------------|:----------------------------|:--------------------------------|:---------------------------------------------------|:-----------------------------------|:----------|-------------------:|
| Molformer-XL     | 0.88±0.07                   | 0.91±0.04                       | 0.8±0.1                                            | 0.90±0.05                          | 0.88±0.08 |                  1 |
| ChemBERTa-2      | 0.87±0.06                   | 0.89±0.05                       | 0.88±0.09                                          | 0.88±0.06                          | 0.88±0.07 |                  2 |
| ECFP-16          | 0.82±0.06                   | 0.84±0.05                       | 0.9±0.1                                            | 0.83±0.06                     

In [17]:
print(tab3.to_markdown(index=False))

| Representation      | Protein-peptide binding affinity   | Antiviral   | Antibacterial   | Cell penetration   | Average   |   Significant rank |
|:--------------------|:-----------------------------------|:------------|:----------------|:-------------------|:----------|-------------------:|
| ChemBERTa-2         | 0.23±0.06                          | 0.23±0.06   | 0.21±0.02       | 0.21±0.02          | 0.22±0.05 |                  1 |
| ECFP-16             | 0.18±0.07                          | 0.18±0.07   | 0.21±0.04       | 0.21±0.04          | 0.19±0.06 |                  2 |
| PeptideCLM          | 0.19±0.04                          | 0.19±0.04   | 0.19±0.04       | 0.19±0.04          | 0.19±0.04 |                  3 |
| Molformer-XL        | 0.16±0.02                          | 0.16±0.02   | 0.15±0.02       | 0.15±0.02          | 0.15±0.02 |                  3 |
| ECFP-16 with counts | 0.13±0.04                          | 0.13±0.04   | 0.13±0.04       | 0.13±0.04          | 0.13